# Data pre-processing

We began by gathering and refining data on earthquakes, volcanic eruptions, and temperature anomalies. We sourced earthquake data and focused on years before the Common Era, converting these years to positive integers for consistency. Similarly, we obtained eruption data and processed the start years in the same manner. For temperature anomalies, we extracted relevant columns and renamed them for clarity. Additionally, we ensured our long-term temperature dataset spanning 800,000 years was properly formatted.

Next, we needed to combine this data. By creating bins, we mapped earthquake and eruption counts to corresponding time periods. This allowed us to visualize the frequency of these events alongside temperature changes, providing a clearer picture of their potential impact on the climate.

We also merged datasets containing information on emissions and electricity generation, calculating the total fossil fuel usage by summing the contributions from coal, gas, and oil. Focusing on data from the year 2000 onwards, we filtered out any entries with zero fossil fuel usage to ensure our analysis was relevant and accurate. With this merged dataset, we were able to analyze the correlation between fossil fuel usage and CO₂ emissions.

At last, the datasets for the figures about temperature anomalies in the last 150 years and electricity generation by source contained irrelevent columns about other topics which were removed to make the data easier to read. These datasets did not require more pre-processing as all the data we needed was present.

The variables of our visualizations: 

Continuous / Interval: Temperature anomaly

Continuous / Ratio: Electricity (in kWh), annual CO2 emissions, total fossil fuel usage per capita

Discrete / Interval: Date (day, month, year), Year

Discrete / Nominal: Country, source of electricity generation

Discrete / Ratio: count of earthquakes, count of eruptions

Download links:

[Temperature anomalies over the last 150 years](https://ourworldindata.org/explorers/climate-change)

[CO2 emissions per capita](https://ourworldindata.org/explorers/co2)

[Per capita electricity source ](https://ourworldindata.org/grapher/per-capita-electricity-source-stacked)

[Volcano eruptions](https://www.kaggle.com/datasets/jessemostipak/volcano-eruptions)

[Earthquake events](https://www.ngdc.noaa.gov/hazel/view/hazards/earthquake/event-data)

[Temperature anomalies over the last 1000 years](https://www.temperaturerecord.org/#sources)


In [1]:
import pandas as pd

# Load DataFrames
earthquakesD = pd.read_csv('../datasets/Earthquakes 2024-06-20.tsv', sep='\t')
earthquakesYears = earthquakesD['Year'][earthquakesD['Year'] < 0].abs().astype(int)

eruptionD = pd.read_csv('../datasets/eruptions.csv')
eruptionYears = eruptionD['start_year'][eruptionD['start_year'] < 0].abs().astype(int)

df = pd.read_csv('../datasets/Temperature dataset 20000 years.csv')
filtered_df = df[['Age (yr BP)', 'Temperature Anomaly (°C) ']].rename(columns={'Age (yr BP)': 'Age', 'Temperature Anomaly (°C)': 'Temperature Anomaly'})

temp800D_df = pd.read_csv('../datasets/Temperature dataset 800,000 years.csv')
temp800D_df.to_csv('../preprocess/pre_Temperature_dataset_800000_years.csv', index=False)

# Combining data by binning
bins = filtered_df['Age']
earthquake_counts = earthquakesYears.value_counts(bins=bins).sort_index()
eruption_counts = eruptionYears.value_counts(bins=bins).sort_index()
filtered_df['Earthquake Counts'] = filtered_df['Age'].map(earthquake_counts).fillna(0).astype(int)
filtered_df['Eruption Counts'] = filtered_df['Age'].map(eruption_counts).fillna(0).astype(int)

filtered_df.to_csv('../preprocess/pre_Temperature_dataset_20000_years.csv', index=False)

In [3]:
df_emissions = pd.read_csv('../datasets/emissions.csv')
df_electricity = pd.read_csv('../datasets/electricity.csv')


merged_df = pd.merge(df_emissions, df_electricity, on=['Entity', 'Code', 'Year'], how='inner')

merged_df['Total Fossil Fuel Usage'] = merged_df[['Coal', 'Gas', 'Oil']].sum(axis=1)

filtered_df = merged_df[(merged_df['Total Fossil Fuel Usage'] > 0) & (merged_df['Year'] >= 2000)]

filtered_df.to_csv('../preprocess/pre_fossil_fuel_emissions.csv', index=False)

correlation = filtered_df[['Total Fossil Fuel Usage', 'Annual CO₂ emissions (per capita)']].corr().loc['Total Fossil Fuel Usage', 'Annual CO₂ emissions (per capita)']
print(f'Correlation between Total Fossil Fuel Usage and CO₂ Emissions: {correlation:.2f}')

Correlation between Total Fossil Fuel Usage and CO₂ Emissions: 0.85
